<a 
href="https://colab.research.google.com/github/5af1/Pioneer-Alpha/blob/master/task3/catr.ipynb"
target="_parent">
<!--- "https://colab.research.google.com/github/saahiluppal/catr/blob/master/catr_demo.ipynb" -->
<img 
src="https://colab.research.google.com/assets/colab-badge.svg" 
alt="Open In Colab"/></a>

<!--- This is an HTML comment in Markdown -->

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! git clone --recursive https://github.com/5AF1/Pioneer-Alpha.git

Cloning into 'Pioneer-Alpha'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 51 (delta 18), reused 29 (delta 0), pack-reused 0
Unpacking objects: 100% (51/51), done.
Submodule 'task3/catr' (https://github.com/5AF1/catr) registered for path 'task3/catr'
Cloning into '/content/Pioneer-Alpha/task3/catr'...
remote: Enumerating objects: 129, done.        
remote: Counting objects: 100% (72/72), done.        
remote: Compressing objects: 100% (36/36), done.        
remote: Total 129 (delta 52), reused 42 (delta 36), pack-reused 57        
Receiving objects: 100% (129/129), 3.05 MiB | 6.43 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Submodule path 'task3/catr': checked out 'b793cd2c2e8dcdad35e2ef870f4498026cbf6f02'


In [3]:
from pathlib import Path
from zipfile import ZipFile
import requests, shutil, os
from PIL import Image
from tqdm import tqdm
import numpy as np
np.random.seed(42)

In [4]:
def my_unzip(archive,output,extract_here = False):
    output = output/archive.stem if not extract_here else output
    
    os.makedirs(output,exist_ok=True)

    with ZipFile(archive, 'r') as zip_ref:
        zip_ref.extractall(output)


In [5]:
content = Path("/content")
drive_shortcut = Path('drive/.shortcut-targets-by-id/10BCfOwDyroU69Nn61LeZic0c9lQVF74a/paper dataset/')
image_zip_file = content/drive_shortcut/'images.zip'

In [6]:
my_unzip(image_zip_file,content/'Pioneer-Alpha/task3/catr',extract_here=True)

In [7]:
import json
def load_data(file_):
    with open(file_, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    return data

def save_data(file_,data):
    os.makedirs(file_.parent,exist_ok=True)
    with open(file_,'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

In [8]:
INFO_DICT = {
        "description": "BanglaLekhaImageCaptions Dataset",
        "url": "https://data.mendeley.com/datasets/rxxch9vw59/2",
        "version": "2.0",
        "year": 2019,
        "contributor": "Nafees Mansoor, Abrar Hasin Kamal, Nabeel Mohammed, Sifat Momen, Md Matiur Rahman",
        "date_created": "2019/07/28"
    }

LICENSES_LIST = []

In [9]:
images_list = []
image_dict = {
            #"license": 2,
            "file_name": "000000000042.jpg",
            #"coco_url": "http://images.cocodataset.org/train2017/000000000042.jpg",
            "height": 478,
            "width": 640,
            #"date_captured": "2013-11-18 09:22:23",
            #   "flickr_url": "http://farm7.staticflickr.com/6024/6016274664_ea4ecac20c_z.jpg",
            "id": 42
}

annotations_list = []
annotations_dict = {
            "image_id": 74,
            "id": 145996,
            "caption": "A picture of a dog laying on the ground."
}

caption_dict = {'caption': ['রাস্তা দিয়ে কতকগুলো শিশু সারিবদ্ধ ভাবে হেঁটে যাচ্ছে।',
   'কিছু বাচ্চা গ্রামের রাস্তা দিয়ে হাতে যাচ্ছে এবং সাথে একজন পুরুষ মানুষ।'],
  'filename': '981.png'}

In [10]:
caption_path = Path('/content/Pioneer-Alpha/task3/bl_cap/captions.json')
caption_data = load_data(caption_path)

In [11]:
np.random.seed(42)
indices = np.arange(len(caption_data))
np.random.shuffle(indices)

split_frac_size = [.7,.2,.1]
split_frac_points = np.cumsum(split_frac_size)
split_frac_points = split_frac_points[:-1]

train_indices, val_indices, test_indices = np.split(indices,[round( len(indices)*frac ) for frac in split_frac_points])
len(train_indices), len(val_indices), len(test_indices)

(6408, 1831, 915)

In [12]:
train_indices

array([8874, 2214, 4112, ..., 4082, 1486, 3678])

In [13]:
train_dict_list = [caption_data[i] for i in train_indices]
val_dict_list = [caption_data[i] for i in val_indices]
test_dict_list = [caption_data[i] for i in test_indices]

In [14]:
def create_annotation_json(data_dict_list,images_path,coco_path,tvt='train'):
    images_list = []
    annotations_list = []

    for data_dict in tqdm(data_dict_list,desc = tvt):
        img_file_name = data_dict['filename']
        caption_list = data_dict['caption']

        id_no = int(img_file_name.split('.')[0])
        img_src = Image.open(images_path/img_file_name)

        image_dict = {
            "file_name": img_file_name.zfill(5+4),
            "height": img_src.height,
            "width": img_src.width,
            "id": id_no
        }

        for i,caption in enumerate(caption_list):
            annotations_dict = {
                "image_id": id_no,
                "id": id_no*1000+i+1,
                "caption": caption
            }
            annotations_list.append(annotations_dict)
        
        images_list.append(image_dict)

        src_path = images_path/img_file_name
        dst_path = coco_path/f'{tvt}2017'/image_dict['file_name']

        os.makedirs(dst_path.parent,exist_ok=True)
        shutil.copy(src_path,dst_path)


    captions_2017 = {
                "info":INFO_DICT,
                "licenses":LICENSES_LIST,
                "images":images_list,
                "annotations":annotations_list,
    }
    captions_2017_path = coco_path/'annotations'/f'captions_{tvt}2017.json'
    save_data(captions_2017_path,captions_2017)



In [15]:
images_path = Path('/content/Pioneer-Alpha/task3/catr/images')
coco_path = Path('/content/Pioneer-Alpha/task3/coco/')

create_annotation_json(train_dict_list,images_path,coco_path,tvt='train')
create_annotation_json(val_dict_list,images_path,coco_path,tvt='val')
create_annotation_json(test_dict_list,images_path,coco_path,tvt='test')

test: 100%|██████████| 915/915 [00:06<00:00, 132.98it/s]


In [ ]:
%cd /content/Pioneer-Alpha/task3/catr

/content/Pioneer-Alpha/task3/catr


In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.4 MB/s 
     |████████████████████████████████| 101 kB 12.2 MB/s 
     |████████████████████████████████| 596 kB 71.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!python predict.py --path .github/cake.png

Downloading: "https://github.com/saahiluppal/catr/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100% 171M/171M [00:02<00:00, 77.5MB/s]
Downloading: "https://github.com/saahiluppal/catr/releases/download/0.2/weight493084032.pth" to /root/.cache/torch/hub/checkpoints/weight493084032.pth
100% 322M/322M [00:03<00:00, 88.3MB/s]
Downloading: 100% 226k/226k [00:00<00:00, 2.70MB/s]
Downloading: 100% 28.0/28.0 [00:00<00:00, 23.6kB/s]
Downloading: 100% 570/570 [00:00<00:00, 589kB/s]
A person cutting a cake with a knife.


In [ ]:
!python main.py

Initializing Device: cuda
Number of params: 83959866
Train: 12815
Valid: 3662
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Start Training..
Epoch: 0
  0% 0/400 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`